In [19]:
import pandas as pd

In [20]:
data=pd.read_csv("home_Data_train.csv")

In [21]:
data.shape

(7239, 244)

In [22]:
X=data.drop(['price'],axis='columns')

In [23]:
X.shape

(7239, 243)

In [24]:
y=data.price

In [25]:
y

0       428.0
1       194.0
2       235.0
3       130.0
4       148.0
        ...  
7234     70.0
7235    200.0
7236    110.0
7237     26.0
7238    400.0
Name: price, Length: 7239, dtype: float64

In [26]:
len(y)

7239

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

In [28]:
from sklearn.linear_model import LinearRegression
model_clf=LinearRegression()
model_clf.fit(X_train,y_train)
model_clf.score(X_test,y_test)

0.8629132245229446

use k fold cross validation to measure accuracy of our linearRegression model

<!-- from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv) -->

In [29]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)



array([0.82702546, 0.86027005, 0.85322178, 0.8436466 , 0.85481502])

In [30]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [31]:
find_best_model_using_gridsearchcv(X,y)

,model,best_score,best_params
0,linear_regression,0.847796,{'normalize': False}
1,lasso,0.726784,"{'alpha': 2, 'selection': 'random'}"
2,decision_tree,0.739833,"{'criterion': 'friedman_mse', 'splitter': 'ran..."


Based on above results we can say that LinearRegression gives the best score. Hence we will use that.


Test the model for few properties


In [34]:
import numpy as np
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return model_clf.predict([x])[0]

In [35]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

83.86570258311637

In [36]:
predict_price("Rajaji Nagar",2800,2,2)

369.99312205862117

Exporat the test model in the pickle file

In [37]:
import pickle

In [ ]:
# with open('banglore_home_prices_model.pickle','wb') as f:
#     pickle.dump(lr_clf,f)

In [38]:
with open("banglore_home_prices_model.pickle",'wb') as f:
    pickle.dump(model_clf,f)

export location and column informaton to a file that will be useful later on
in our prediction application

In [39]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))